In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.sparse import csr_matrix, issparse


In [2]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Performing Abstract Summarization Task

In [4]:
text_to_summarize ="""
Niket Girdhar is a deep learning enthusiast, researcher, and developer based in India, with a keen focus on computer vision, natural language processing, and generative AI. Having worked extensively on projects involving GANs, YOLO object detection, and transformer-based models, Niket blends strong technical foundations with practical experimentation. He has a particular interest in refining and benchmarking model performance, from tuning MIRNet for image denoising to fine-tuning BERT-like models for tone-controlled LinkedIn post generation. Alongside research papers and IEEE-format literature reviews, he also explores creative AI-driven workflows like Reddit scraping and LLM post synthesis. Whether working on training pipelines, writing scientific summaries, or crafting detailed visualizations, Niket approaches every project with analytical rigor and a drive for innovation.
"""

preprocess_text = text_to_summarize.strip().replace("\n","")

print ("original text preprocessed:\n", preprocess_text)

original text preprocessed:
 Niket Girdhar is a deep learning enthusiast, researcher, and developer based in India, with a keen focus on computer vision, natural language processing, and generative AI. Having worked extensively on projects involving GANs, YOLO object detection, and transformer-based models, Niket blends strong technical foundations with practical experimentation. He has a particular interest in refining and benchmarking model performance, from tuning MIRNet for image denoising to fine-tuning BERT-like models for tone-controlled LinkedIn post generation. Alongside research papers and IEEE-format literature reviews, he also explores creative AI-driven workflows like Reddit scraping and LLM post synthesis. Whether working on training pipelines, writing scientific summaries, or crafting detailed visualizations, Niket approaches every project with analytical rigor and a drive for innovation.


In [5]:
t5_prepared_text = "summarize: " + preprocess_text

input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

summary_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    min_length=30,
    max_length=50,
    early_stopping=True
)

output = base_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print (f"Summarized text: \n{output}")

Summarized text: 
Niket Girdhar is a deep learning enthusiast, researcher, and developer based in india . he has a keen focus on computer vision, natural language processing, and generative AI . Niket blends strong technical


## Language Translation: English-to-German

In [6]:
input_ids = base_tokenizer.encode('translate English to German: Where is the chocolate?', return_tensors='pt')

translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print (f"Translated text:\n{output}")

Translated text:
Wo ist die Schokolade?


In [7]:
input_ids = base_tokenizer('translate English to German: Where is the chocolate?', return_tensors='pt').input_ids
labels = base_tokenizer('Wo ist die Schokolade?', return_tensors='pt').input_ids # pass labels in to calculate loss

loss = base_model(input_ids=input_ids, labels=labels).loss

labels, loss

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


(tensor([[ 3488,   229,    67, 31267,    58,     1]]),
 tensor(0.1136, grad_fn=<NllLossBackward0>))

## CoLA: The Corpus of Linguistic Acceptability

checking for grammatical correctess

In [8]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

is grammatically correct?: 
acceptable


In [9]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

translate_ids = base_model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


is grammatically correct?: 
unacceptable


## STSB: Semantic Text Similarity Benchmark

Are two sentences semantically similar

In [10]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


semantically similar? (0-5): 
3.2


In [11]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


semantically similar? (0-5): 
0.4


## MNLI - Multi-Genre Natural Language Inference

Whether a premise implies (“entailment”), contradicts (“contradiction”), or neither (“neutral”) a hypothesis.

In [3]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I am running for mayor', return_tensors='pt'
)

translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Response: 
entailment


In [4]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I do not really vote', return_tensors='pt'
)

translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Response: 
contradiction


In [5]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I code for a living', return_tensors='pt'
)

translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Response: 
neutral


## Question/Answering

In [8]:
input_ids = base_tokenizer.encode(
    'question: Where does Niket live? context: Niket lives in Chennai but he is originally from Jalalabad West.', return_tensors='pt'
)

translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Response: 
Chennai


In [9]:
input_ids = base_tokenizer.encode(
    'question: Where does Niket live? context: Niket lives in Chennai but Nitish lives in Delhi.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Response: 
Chennai
